# TSA Chapter 10: Rolling vs Multi-Step GARCH Forecasts

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch10/TSA_ch10_rolling_vs_multistep/TSA_ch10_rolling_vs_multistep.ipynb)

This notebook demonstrates:
- Comparing rolling one-step-ahead GARCH forecasts with multi-step forecasts for Bitcoin volatility.

In [ ]:
!pip install yfinance arch matplotlib numpy pandas -q

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import yfinance as yf
from arch import arch_model
import os, warnings
warnings.filterwarnings('ignore')

In [ ]:
# Color scheme and style configuration
COLORS = {
    'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32',
    'orange': '#E67E22', 'gray': '#666666', 'purple': '#8E44AD',
}

plt.rcParams.update({
    'figure.facecolor': 'none', 'axes.facecolor': 'none',
    'savefig.facecolor': 'none', 'savefig.transparent': True,
    'axes.spines.top': False, 'axes.spines.right': False,
    'axes.grid': False, 'font.size': 10, 'axes.titlesize': 12,
    'axes.labelsize': 10, 'xtick.labelsize': 9, 'ytick.labelsize': 9,
    'legend.fontsize': 9, 'figure.dpi': 150, 'lines.linewidth': 1.2,
    'axes.linewidth': 0.6, 'legend.facecolor': 'none',
    'legend.framealpha': 0, 'legend.edgecolor': 'none',
})

def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    try:
        charts_path = os.path.join('..', '..', '..', 'charts', name)
        fig.savefig(f'{charts_path}.pdf', bbox_inches='tight', transparent=True, dpi=150)
        fig.savefig(f'{charts_path}.png', bbox_inches='tight', transparent=True, dpi=150)
    except Exception:
        pass
    print(f'Saved: {name}.pdf + .png')

def legend_outside(ax, ncol=3, y=-0.18):
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, y), ncol=ncol, frameon=False)

In [ ]:
# Download Bitcoin data
btc = yf.download('BTC-USD', start='2019-01-01', progress=False)
if isinstance(btc.columns, pd.MultiIndex):
    btc.columns = btc.columns.get_level_values(0)
btc['Return'] = btc['Close'].pct_change() * 100
btc.dropna(inplace=True)
print(f'BTC data: {len(btc)} daily observations')

# Rolling vs multi-step GARCH forecasts
returns = btc['Return'].dropna()
n_test = 120
n_train = len(returns) - n_test

# Rolling one-step (subsample for speed)
step = max(1, n_test // 30)
rolling_vol, rolling_idx = [], []
for i in range(0, n_test, step):
    window = returns.iloc[:n_train + i]
    mod = arch_model(window, vol='Garch', p=1, q=1, mean='Constant', dist='normal')
    r = mod.fit(disp='off', update_freq=0)
    fcast = r.forecast(horizon=1)
    rolling_vol.append(np.sqrt(fcast.variance.values[-1, 0]))
    rolling_idx.append(returns.index[n_train + i])

# Multi-step from single point
window_ms = returns.iloc[:n_train]
mod_ms = arch_model(window_ms, vol='Garch', p=1, q=1, mean='Constant', dist='normal')
r_ms = mod_ms.fit(disp='off')
fcast_ms = r_ms.forecast(horizon=n_test)
multistep_vol = np.sqrt(fcast_ms.variance.values[-1, :])

test_idx = returns.index[n_train:n_train + n_test]
realized = np.abs(returns.iloc[n_train:n_train + n_test].values)
realized_smooth = pd.Series(realized, index=test_idx).rolling(10).mean()

fig, axes = plt.subplots(1, 2, figsize=(10, 3.5), sharey=True)

axes[0].plot(rolling_idx, rolling_vol, color=COLORS['blue'], lw=0.9, label='Rolling 1-step')
axes[0].plot(test_idx, realized_smooth, color=COLORS['gray'], lw=0.7, alpha=0.6, label='$|r_t|$ (10d MA)')
axes[0].set_title('(a) Rolling One-Step Forecast')
axes[0].set_ylabel('Volatility (%)')
axes[0].xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
plt.setp(axes[0].get_xticklabels(), rotation=30, ha='right')
axes[0].legend(loc='upper center', bbox_to_anchor=(0.5, -0.22), ncol=2, frameon=False)

axes[1].plot(test_idx, multistep_vol, color=COLORS['red'], lw=0.9, label='Multi-step')
axes[1].plot(test_idx, realized_smooth, color=COLORS['gray'], lw=0.7, alpha=0.6, label='$|r_t|$ (10d MA)')
axes[1].set_title('(b) Multi-Step Forecast')
axes[1].xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
plt.setp(axes[1].get_xticklabels(), rotation=30, ha='right')
axes[1].legend(loc='upper center', bbox_to_anchor=(0.5, -0.22), ncol=2, frameon=False)

fig.suptitle('Rolling vs Multi-Step GARCH Forecasts', y=1.02, fontsize=11)
fig.tight_layout()
save_chart(fig, 'rolling_vs_multistep')
plt.show()